# 2-4 토큰화하기

## 코랩 노트북 초기화하기

의존성 패키지 설치

In [ ]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 766 kB/s 
     |████████████████████████████████| 806 kB 9.6 MB/s 
     |████████████████████████████████| 2.8 MB 47.0 MB/s 
     |████████████████████████████████| 57 kB 4.0 MB/s 
     |████████████████████████████████| 332 kB 50.6 MB/s 
     |████████████████████████████████| 132 kB 49.4 MB/s 
     |████████████████████████████████| 636 kB 44.8 MB/s 
     |████████████████████████████████| 829 kB 44.7 MB/s 
     |████████████████████████████████| 61 kB 420 kB/s 
     |████████████████████████████████| 3.3 MB 43.6 MB/s 
     |████████████████████████████████| 895 kB 66.5 MB/s 
     |████████████████████████████████| 1.1 MB 56.7 MB/s 
     |████████████████████████████████| 96 kB 5.4 MB/s 
     |████████████████████████████████| 192 kB 53.0 MB/s 
     |████████████████████████████████| 160 kB 65.3 MB/s 
     |████████████████████████████████| 271 kB 49.1 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 s

구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


## GPT 입력값 만들기

2-3에서 만든 바이트 기준 BPE 어휘 집합과 바이그램 쌍의 병합 우선순위 파일 필요

GPT 토크나이저 선언

In [ ]:
from transformers import GPT2Tokenizer

tokenizer_gpt = GPT2Tokenizer.from_pretrained("/gdrive/My Drive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file /gdrive/My Drive/nlpbook/bbpe/config.json not found


GPT 토크나이저로 토큰화하기

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

In [ ]:
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĶ',
  'ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

GPT 모델 입력 만들기

In [ ]:
batch_inputs = tokenizer_gpt(
    sentences,
    padding="max_length", # 문장의 최대 길이에 맞춰 패딩
    max_length=12, # 문장의 토큰 기준 최대 길이
    truncation=True, # 문장 잘림 허용 옵셥
)

In [ ]:
batch_inputs.keys()

dict_keys(['input_ids', 'attention_mask'])

인덱싱(indexing) : 각 토큰을 인덱스로 변환하는 과정

In [ ]:
batch_inputs['input_ids'] # 토큰화 결과를 가지고 각 토큰을 인덱스로 바꾼것
# 0 : [PAD] 토큰에 해당하는 값. 일종의 더미 토큰으로 길이를 맞춰주는 역할을 함
# 문장2는 15->12로 문장의 길이가 줄어듦(truncation=True)

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 758, 2883, 356, 806, 422, 9875, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
batch_inputs['attention_mask'] # 일반 토큰(1)과 패딩 토큰(0)의 위치를 구분해 알려줌

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

## BERT 입력값 만들기

BERT 토크나이저 선언

In [ ]:
from transformers import BertTokenizer

tokenizer_bert = BertTokenizer.from_pretrained(
    "/gdrive/My Drive/nlpbook/wordpiece",
    do_lower_case=False,
)

file /gdrive/My Drive/nlpbook/wordpiece/config.json not found


BERT 토크나이저로 토큰화하기

In [ ]:
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

In [30]:
tokenized_sentences
# ##네요처럼 ##은 해당 토큰이 띄어쓰기 기준 어절의 시작이 아님을 나타냄

[['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리'],
 ['흠',
  '.',
  '.',
  '.',
  '포스터',
  '##보고',
  '초딩',
  '##영화',
  '##줄',
  '.',
  '.',
  '.',
  '.',
  '오버',
  '##연기',
  '##조차',
  '가볍',
  '##지',
  '않',
  '##구나'],
 ['별루', '였다', '.', '.']]

BERT 모델 입력 만들기

In [ ]:
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

In [ ]:
batch_inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
batch_inputs['input_ids']
# 2 : 문장의 시작을 나타내는 [CLS] 토큰
# 3 : 문장의 끝을 나타내는 [SEP] 토큰

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 1208, 16, 3],
 [2, 3274, 9508, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

In [ ]:
batch_inputs['attention_mask']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [ ]:
batch_inputs['token_type_ids'] # segment 정보
# BERT 모델은 기본적으로 문서 또는 문장 2개를 입력받는데 이 둘을 token_type_ids로 구분함(0, 1)
# 우리는 문장을 하나씩 넣었으므로 모두 0

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]